<a href="https://colab.research.google.com/github/harnalashok/classification/blob/main/springleaf_reduce_mem_size.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Springleaf problem--Reducing dataset size
This notebook shows:<br>
a) How to reduce memory size of a DataFrame. <br>
b. How to derive correct dtype information of each column in a dictionary format<br>
c. How to use this dictionary to read the large dataset in correct dtype format<br>
d. How to store the dtype-dictioary for use subsequently<br>


In [150]:
# 1.0 Import libraries
import pandas as pd
import numpy as np
import os

In [151]:
# 1.1 Display multiple commands output from a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


### Mount gdrive

In [152]:
# 2.0
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


### Read data

In [154]:
# 2.1 Read train.csv.zip file from gdrive
#     Takes 2 minutes

%%time
filepath = "/gdrive/MyDrive/Colab_data_files/springleaf_marketing/train.csv.zip"
df = pd.read_csv(filepath, low_memory = False)

CPU times: user 1min 49s, sys: 23.9 s, total: 2min 13s
Wall time: 2min 14s


In [155]:
# 2.2 Memory usage?
df.shape
print()
df.memory_usage().sum()  # 22,470,14160 bytes

(145231, 1934)

2247014160

### Get column names where nulls do not exist
Else, null columns must first be filled in before memory can be reduced.

In [ ]:
# 2.3 Which columns contain nulls
cols = df.isnull().sum()[df.isnull().sum() > 0]
print(cols.index.values)

In [ ]:
# 2.4 Array of column names that are NOT null
df.isnull().sum()[df.isnull().sum() > 0].index.values

In [159]:
# 2.5 List of column that have no nulls
cols = list(df.isnull().sum()[df.isnull().sum() <= 0].index.values)
print(cols)

['ID', 'VAR_0001', 'VAR_0002', 'VAR_0003', 'VAR_0004', 'VAR_0005', 'VAR_0532', 'VAR_0533', 'VAR_0534', 'VAR_0535', 'VAR_0536', 'VAR_0537', 'VAR_0538', 'VAR_0539', 'VAR_0540', 'VAR_0541', 'VAR_0542', 'VAR_0543', 'VAR_0544', 'VAR_0545', 'VAR_0546', 'VAR_0547', 'VAR_0548', 'VAR_0549', 'VAR_0550', 'VAR_0551', 'VAR_0552', 'VAR_0553', 'VAR_0554', 'VAR_0555', 'VAR_0556', 'VAR_0557', 'VAR_0558', 'VAR_0559', 'VAR_0560', 'VAR_0561', 'VAR_0562', 'VAR_0563', 'VAR_0564', 'VAR_0565', 'VAR_0566', 'VAR_0567', 'VAR_0568', 'VAR_0569', 'VAR_0570', 'VAR_0571', 'VAR_0572', 'VAR_0573', 'VAR_0574', 'VAR_0575', 'VAR_0576', 'VAR_0577', 'VAR_0578', 'VAR_0579', 'VAR_0580', 'VAR_0581', 'VAR_0582', 'VAR_0583', 'VAR_0584', 'VAR_0585', 'VAR_0586', 'VAR_0587', 'VAR_0588', 'VAR_0589', 'VAR_0590', 'VAR_0591', 'VAR_0592', 'VAR_0593', 'VAR_0594', 'VAR_0595', 'VAR_0596', 'VAR_0597', 'VAR_0598', 'VAR_0599', 'VAR_0600', 'VAR_0601', 'VAR_0602', 'VAR_0603', 'VAR_0604', 'VAR_0605', 'VAR_0606', 'VAR_0607', 'VAR_0608', 'VAR_0609

The following function has bee taken from Kaggle. It has been modified to return both the dataframe (with reduced memory) as also column types. 

In [160]:
# 3.0 Last amended: 20th August, 2021
# Myfolder: C:\Users\Administrator\OneDrive\Documents\useful_code & utilities
# Ref: https://www.kaggle.com/gemartin/load-data-reduce-memory-usage

# 3.1 Will store column types
coltypes = {}

# 3.2 Function to reduce memory
#     DataFrame must not have NULLs

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage. 
        If NaN exist fill them up first		
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                    coltypes[col] = np.int8
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                    coltypes[col] = np.int16
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                    coltypes[col] = np.int32
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64) 
                    coltypes[col] = np.int64  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                    coltypes[col] = np.float16
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                    coltypes[col] = np.float32
                else:
                    df[col] = df[col].astype(np.float64)
                    coltypes[col] = np.float64
        else:
            df[col] = df[col].astype('category')
            coltypes[col] = 'category'

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df,coltypes

In [161]:
# 4.0 Reduce memory usage
#      For not null columns
#       7 mins

%%time

# 4.1 Consider only not null columns
data = df[cols].copy()

# 4.2 Apply reduce_mem_usage()
data,feat_types = reduce_mem_usage(data)

Memory usage of dataframe is 1561.21 MB
Memory usage after optimization is: 429.91 MB
Decreased by 72.5%
CPU times: user 7min 8s, sys: 44.8 s, total: 7min 53s
Wall time: 7min 51s


In [162]:
# 4.3
data.shape
print()

# 4.3.1
data.memory_usage().sum()   # 450797648
print()

# 4.3.2
print("\nColumn wise Data types:")
print(feat_types)

print("\nHow many columns?")
len(feat_types)    # 1409

(145231, 1409)

450797648



Column wise Data types:
{'ID': <class 'numpy.int32'>, 'VAR_0001': 'category', 'VAR_0002': <class 'numpy.int16'>, 'VAR_0003': <class 'numpy.int16'>, 'VAR_0004': <class 'numpy.int32'>, 'VAR_0005': 'category', 'VAR_0532': <class 'numpy.int8'>, 'VAR_0533': <class 'numpy.int8'>, 'VAR_0534': <class 'numpy.int8'>, 'VAR_0535': <class 'numpy.int8'>, 'VAR_0536': <class 'numpy.int8'>, 'VAR_0537': <class 'numpy.int8'>, 'VAR_0538': <class 'numpy.int8'>, 'VAR_0539': <class 'numpy.int8'>, 'VAR_0540': <class 'numpy.int8'>, 'VAR_0541': <class 'numpy.int32'>, 'VAR_0542': <class 'numpy.int32'>, 'VAR_0543': <class 'numpy.int32'>, 'VAR_0544': <class 'numpy.int32'>, 'VAR_0545': <class 'numpy.int16'>, 'VAR_0546': <class 'numpy.int16'>, 'VAR_0547': <class 'numpy.int16'>, 'VAR_0548': <class 'numpy.int16'>, 'VAR_0549': <class 'numpy.int16'>, 'VAR_0550': <class 'numpy.int16'>, 'VAR_0551': <class 'numpy.int16'>, 'VAR_0552': <class 'numpy.int8'>, 'VAR_0553': <class 'numpy.int8'>, 'VAR_0554': <class 'numpy.int16'

1409

In [163]:
%%time

# 5.0 Now modify the original dataframe
df[cols] = data   # 7 minutes

# 5.1 And delete data
del data

CPU times: user 7min 7s, sys: 44.1 s, total: 7min 51s
Wall time: 7min 49s


In [164]:
# 5.2
df.shape
print()
df.memory_usage().sum()  # 1060767848

(145231, 1934)

1060767848

In [ ]:
feat_types

In [85]:
# 6.0 Next time:
# 6.0 Read dataset again but specify
#     column dtypes. 
#     Columns whose dtypes have not been specified
#     will be read by default dtype of 64 bit

%%time

dfx = pd.read_csv(filepath,
                  dtype = feat_types
                  )


<string>:2: DtypeWarning: Columns (8,9,10,11,12,43,157,196,214,225,228,229,231,235,238) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 36 s, sys: 4.88 s, total: 40.8 s
Wall time: 41 s


In [87]:
# 6.1 Checks:
dfx.memory_usage().sum()
print()
dfx.shape

1060767848

(145231, 1934)

### You can save your dictioary to gdrive as follows:
Generally 'pickle' package is used to store python objects. We will use pickle here also.


In [177]:
# 7.0 Define functions to save dictionary to a file
#      and read from it
# Ref StackOverflow
#       https://stackoverflow.com/a/19201448/3282777

# 7.1 Import needed package
import pickle

# 7.2 Function to save the dictionary (ie obj)
def save_obj(obj, folder, fileToSaveTo ):
    with open(folder+'/'+ fileToSaveTo + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


# 7.3 Read the dictionary ie stored pickle file
def load_obj(folder, pickle_file ):
    with open(folder +'/' + pickle_file + '.pkl', 'rb') as f:
        return pickle.load(f)

In [179]:
# 8.0 First create a blank file where we want
#      our dictionary containing dtype information
#        to be stored

# 8.1 The folder and filename
folder = "/gdrive/MyDrive/Colab_data_files/springleaf_marketing"
fileToSaveTo = "myfile"

# 8.2 Make the folder current folder
os.chdir(folder)

# 8.3 Create a file of '0 byte' size
!touch myfile.pkl

In [176]:
# 8.4 Save dictioary to file
save_obj(feat_types, folder, fileToSaveTo)

In [ ]:
# 9.0 Retireve the dictioary back
#     For use in reading dataset
load_obj(folder, "myfile")

In [ ]:
############# I am done ##############